In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import geemap
import ee
import os
import pandas as pd
from coastsat.coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects
import shapely
import pickle

from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

try:
    ee.Initialize()

except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Region of interest

In [2]:
# Channel
polygon = [[[166.908527, -0.539382],
            [166.908527, -0.542476],
            [166.912789, -0.542476],
            [166.912789, -0.539382],
            [166.908527, -0.539382]]]

# Airport
polygon_airport = [[[166.908527, -0.536727],
            [166.908527, -0.558934],
            [166.928538, -0.558934],
            [166.928538, -0.536727],
            [166.908527, -0.536727]]]

polygon_coast = [[[166.905418, -0.528960], 
            [166.905418, -0.540194],
            [166.915109, -0.540194],
            [166.915109, -0.528960],
            [166.905418, -0.528960]]]

polygon_Maldives = [[[73.479003, 0.610048], 
            [73.479003, 0.598734],
            [73.493267, 0.598734],
            [73.493267, 0.610048],
            [73.479003, 0.610048]]]

polygon_gee = ee.Geometry.Polygon(polygon_Maldives) 

In [3]:
# date range
dates = ['2013-01-01', '2023-12-31']

# satellite missions 
sat_list = ['L5','L7','L8','L9','S2']
sat_list = ['L8', 'L9', 'S2']

# choose Landsat collection 'C01' or 'C02'
collection = 'C02'

# name of the site
sitename = 'Vodamulaa_Maldives_2013_2023'

# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data\\coastsat')

# put all the inputs into a dictionnary
inputs = {'polygon': polygon_Maldives, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath': filepath,
         'landsat_collection': collection}

# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Number of images available between 2013-01-01 and 2023-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 297 images
     L9: 43 images
     S2: 434 images
  Total to download: 774 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L8: 111 images
  Total Tier 2: 111 images


In [27]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'dist_clouds': 50,         # ditance around clouds where shoreline can't be mapped
    'output_epsg': 32643,       # epsg code of spatial reference system desired for the output
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 0,     # minimum area (in metres^2) for an object to be labelled as a beach
    'min_length_sl': 0,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': True,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'latest', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off': False,           # True to switch pansharpening off for Landsat 7/8/9 imagery
    # add the inputs defined previously
    'inputs': inputs,
}

In [5]:
metadata = SDS_download.get_metadata(inputs) 

In [6]:
SDS_preprocess.save_jpg(metadata, settings)

Saving images as jpg:
L8: 296 images
100%
L9: 42 images
100%
S2: 433 images
92%

c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\coastsat\coastsat\SDS_preprocess.py:534: UserWarning: c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023\jpg_files\preprocessed\RGB\2022-11-07-05-38-50_RGB_S2.jpg is a low contrast image
  imsave(fname, im_RGB, quality=100)
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\coastsat\coastsat\SDS_preprocess.py:534: UserWarning: c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023\jpg_files\preprocessed\RGB\2022-11-07-05-38-56_RGB_S2.jpg is a low contrast image
  imsave(fname, im_RGB, quality=100)


100%
Satellite images saved as .jpg in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023\jpg_files\preprocessed


In [5]:
metadata = SDS_download.retrieve_images(inputs) 

Number of images available between 2013-01-01 and 2023-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     L8: 296 images
     L9: 42 images
     S2: 433 images
  Total to download: 771 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L8: 110 images
  Total Tier 2: 110 images

L8: 296 images
100%
L9: 42 images
100%
S2: 433 images
100%
Satellite images downloaded from GEE and save in c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023


In [28]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 25 # max distance (in meters) allowed from the reference shoreline

Reference shoreline already exists and was loaded
Reference shoreline coordinates are in epsg:32643


In [29]:
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

Mapping shorelines:
L8:   1%

c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


L8:   92%Could not map shoreline for this image: 2022-05-31-05-20-37_L8_Vodamulaa_Maldives_2013_2023_ms.tif
L8:   100%
L9:   2%

c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


L9:   100%
S2:   0%

c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


S2:   100%


In [30]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

112 duplicates
17 bad georef


In [31]:
from pyproj import CRS
geomtype = 'points' # choose 'points' or 'lines' for the layer geometry
gdf = SDS_tools.output_to_gdf(output, geomtype)
if gdf is None:
    raise Exception("output does not contain any mapped shorelines")
gdf.crs = CRS(settings['output_epsg']) # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'], '%s_output_%s.geojson'%(sitename,geomtype)),
                                driver='GeoJSON', encoding='utf-8')

In [32]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()
plt.show();

KeyboardInterrupt: 

In [33]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f)
# remove duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_duplicates(output)
# remove inaccurate georeferencing (set threshold to 10 m)
output = SDS_tools.remove_inaccurate_georef(output, 10)

112 duplicates
17 bad georef


In [35]:
%matplotlib qt
#transects = SDS_transects.draw_transects(output, settings)
geojson_file = os.path.join(os.getcwd(), 'data\\coastsat\\Vodamulaa_Maldives_2013_2023', 'Vodamulaa_Maldives_2013_2023_transects.geojson')
transects = SDS_tools.transects_from_geojson(geojson_file)

51 transects have been loaded coordinates are in epsg:4326


In [36]:
fig = plt.figure(figsize=[15,8], tight_layout=True)
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
for i,key in enumerate(list(transects.keys())):
    plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
    plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
    plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
                va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))

In [37]:
# along-shore distance over which to consider shoreline points to compute the median intersection
settings_transects = {'along_dist':25}
cross_distance = SDS_transects.compute_intersection(output, transects, settings_transects) 

In [20]:
settings_transects = { # parameters for computing intersections
                      'along_dist':          25,        # along-shore distance to use for computing the intersection
                      'min_points':          3,         # minimum number of shoreline points to calculate an intersection
                      'max_std':             15,        # max std for points around transect
                      'max_range':           30,        # max range for points around transect
                      'min_chainage':        -100,      # largest negative value along transect (landwards of transect origin)
                      'multiple_inter':      'auto',    # mode for removing outliers ('auto', 'nan', 'max')
                      'prc_multiple':         0.1,      # percentage of the time that multiple intersects are present to use the max
                     }
cross_distance = SDS_transects.compute_intersection_QC(output, transects, settings_transects) 

c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\numpy\lib\nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\numpy\lib\nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\coastsat\coastsat\SDS_transects.py:337: RuntimeWarning: All-NaN slice encountered
  max_intersect[i] = np.nanmax(xy_rot[0,:])
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\coastsat\coastsat\SDS_transects.py:338: RuntimeWarning: All-NaN slice encountered
  min_intersect[i] = np.nanmin(xy_rot[0,:])


In [38]:
from matplotlib import gridspec
fig = plt.figure(figsize=[15,8], tight_layout=True)
gs = gridspec.GridSpec(len(cross_distance),1)
gs.update(left=0.05, right=0.95, bottom=0.05, top=0.95, hspace=0.05)
for i,key in enumerate(cross_distance.keys()):
    if np.all(np.isnan(cross_distance[key])):
        continue
    ax = fig.add_subplot(gs[i,0])
    ax.grid(linestyle=':', color='0.5')
    ax.set_ylim([-50,50])
    ax.plot(output['dates'], cross_distance[key]- np.nanmedian(cross_distance[key]), '-o', ms=6, mfc='w')
    ax.set_ylabel('distance [m]', fontsize=12)
    ax.text(0.5,0.95, key, bbox=dict(boxstyle="square", ec='k',fc='w'), ha='center',
            va='top', transform=ax.transAxes, fontsize=14)  

c:\Users\myriampe\anaconda3\envs\coastsatn\lib\site-packages\ipykernel\eventloops.py:128: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [39]:
# save a .csv file for Excel users
out_dict = dict([])
out_dict['dates'] = output['dates']
for key in transects.keys():
    out_dict[key] = cross_distance[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'transect_time_series_May29.csv')
df.to_csv(fn, sep=',')
print('Time-series of the shoreline change along the transects saved as:\n%s'%fn)

Time-series of the shoreline change along the transects saved as:
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023\transect_time_series_May29.csv


In [40]:
# remove outliers in the time-series (coastal despiking)
settings_outliers = {'max_cross_change':   40,             # maximum cross-shore change observable between consecutive timesteps
                     'otsu_threshold':     [-.5,0],        # min and max intensity threshold use for contouring the shoreline
                     'plot_fig':           True,           # whether to plot the intermediate steps
                    }
cross_distance = SDS_transects.reject_outliers(cross_distance,output,settings_outliers)

1  - outliers removed: 10
2  - outliers removed: 10
3  - outliers removed: 9
4  - outliers removed: 10
5  - outliers removed: 9
6  - outliers removed: 10
7  - outliers removed: 8
8  - outliers removed: 8
9  - outliers removed: 9
10  - outliers removed: 10
11  - outliers removed: 9
12  - outliers removed: 9
13  - outliers removed: 11
14  - outliers removed: 10
15  - outliers removed: 9
16  - outliers removed: 9
17  - outliers removed: 8
18  - outliers removed: 8
19  - outliers removed: 9
20  - outliers removed: 9
21  - outliers removed: 9


c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\coastsat\coastsat\SDS_transects.py:458: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig,ax=plt.subplots(2,1,figsize=[12,6], sharex=True)


22  - outliers removed: 9
23  - outliers removed: 9
24  - outliers removed: 9
25  - outliers removed: 9
26  - outliers removed: 9
27  - outliers removed: 9
28  - outliers removed: 8
29  - outliers removed: 9
30  - outliers removed: 9
31  - outliers removed: 10
32  - outliers removed: 9
33  - outliers removed: 12
34  - outliers removed: 10
35  - outliers removed: 9
36  - outliers removed: 9
37  - outliers removed: 10
38  - outliers removed: 10
39  - outliers removed: 8
40  - outliers removed: 8
41  - outliers removed: 10
42  - outliers removed: 10
43  - outliers removed: 10
44  - outliers removed: 10
45  - outliers removed: 10
46  - outliers removed: 9
47  - outliers removed: 10
48  - outliers removed: 9
49  - outliers removed: 10
50  - outliers removed: 10
51  - outliers removed: 8


In [41]:
# save a .csv file for Excel users
out_dict = dict([])
out_dict['dates'] = output['dates']
for key in transects.keys():
    out_dict[key] = cross_distance[key]
df = pd.DataFrame(out_dict)
fn = os.path.join(settings['inputs']['filepath'],settings['inputs']['sitename'],
                  'transect_time_series_corrected_29May.csv')
df.to_csv(fn, sep=',')
print('Time-series of the shoreline change along the transects saved as:\n%s'%fn)

Time-series of the shoreline change along the transects saved as:
c:\Users\myriampe\OneDrive\Documents\PhD in Earth Science\Projects\Islands-CI\data\coastsat\Vodamulaa_Maldives_2013_2023\transect_time_series_corrected_29May.csv


In [42]:
season_colors = {'DJF':'C3', 'MAM':'C1', 'JJA':'C2', 'SON':'C0'}
for key in cross_distance.keys():
    chainage = cross_distance[key]
    # remove nans
    idx_nan = np.isnan(chainage)
    dates_nonan = [dates[_] for _ in np.where(~idx_nan)[0]]
    chainage = chainage[~idx_nan] 
    
    # compute shoreline seasonal averages (DJF, MAM, JJA, SON)
    dict_seas, dates_seas, chainage_seas, list_seas = SDS_transects.seasonal_average(dates_nonan, chainage)
    
    # plot seasonal averages
    fig,ax=plt.subplots(1,1,figsize=[14,4],tight_layout=True)
    #ax.grid(b=True,which='major', linestyle=':', color='0.5')
    ax.set_title('Time-series at %s'%key, x=0, ha='left')
    ax.set(ylabel='distance [m]')
    ax.plot(dates_nonan, chainage,'+', lw=1, color='k', mfc='w', ms=4, alpha=0.5,label='raw datapoints')
    ax.plot(dates_seas, chainage_seas, '-', lw=1, color='k', mfc='w', ms=4, label='seasonally-averaged')
    for k,seas in enumerate(dict_seas.keys()):
        ax.plot(dict_seas[seas]['dates'], dict_seas[seas]['chainages'],
                 'o', mec='k', color=season_colors[seas], label=seas,ms=5)
    ax.legend(loc='lower left',ncol=6,markerscale=1.5,frameon=True,edgecolor='k',columnspacing=1)

IndexError: list index out of range

In [43]:
month_colors = plt.cm.get_cmap('tab20')
for key in cross_distance.keys():
    chainage = cross_distance[key]
    # remove nans
    idx_nan = np.isnan(chainage)
    dates_nonan = [dates[_] for _ in np.where(~idx_nan)[0]]
    chainage = chainage[~idx_nan] 
    
    # compute shoreline monthly averages
    dict_month, dates_month, chainage_month, list_month = SDS_transects.monthly_average(dates_nonan, chainage)
    
    # plot monthly averages
    fig,ax=plt.subplots(1,1,figsize=[14,4],tight_layout=True)
    #ax.grid(b=True,which='major', linestyle=':', color='0.5')
    ax.set_title('Time-series at %s'%key, x=0, ha='left')
    ax.set(ylabel='distance [m]')
    ax.plot(dates_nonan, chainage,'+', lw=1, color='k', mfc='w', ms=4, alpha=0.5,label='raw datapoints')
    ax.plot(dates_month, chainage_month, '-', lw=1, color='k', mfc='w', ms=4, label='monthly-averaged')
    for k,month in enumerate(dict_month.keys()):
        ax.plot(dict_month[month]['dates'], dict_month[month]['chainages'],
                 'o', mec='k', color=month_colors(k), label=month,ms=5)
    ax.legend(loc='lower left',ncol=7,markerscale=1.5,frameon=True,edgecolor='k',columnspacing=1)

C:\Users\myriampe\AppData\Local\Temp\ipykernel_23172\1729604929.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  month_colors = plt.cm.get_cmap('tab20')


IndexError: list index out of range

In [50]:
timeseries = pd.read_csv(os.getcwd()+'\\data\\coastsat\\Vodamulaa_Maldives_2013_2023\\transect_time_series_corrected_29May.csv')
plt.figure()
plt.plot(timeseries.dates, timeseries['18'])
plt.show()

In [54]:
for key in cross_distance.keys():
    chainage = cross_distance[key]
    # remove nans
    idx_nan = np.isnan(chainage)
    dates_nonan = [dates[_] for _ in np.where(~idx_nan)[0]]
    chainage = chainage[~idx_nan] 

plt.plot(dates_nonan, chainage)
plt.show()

[datetime.datetime(2013, 12, 16, 5, 22, 9, tzinfo=<UTC>),
 datetime.datetime(2014, 2, 2, 5, 21, 17, tzinfo=<UTC>),
 datetime.datetime(2014, 3, 6, 5, 20, 48, tzinfo=<UTC>),
 datetime.datetime(2014, 4, 7, 5, 20, 46, tzinfo=<UTC>),
 datetime.datetime(2014, 6, 26, 5, 20, 20, tzinfo=<UTC>),
 datetime.datetime(2014, 11, 17, 5, 20, 26, tzinfo=<UTC>),
 datetime.datetime(2015, 2, 5, 5, 20, 10, tzinfo=<UTC>),
 datetime.datetime(2015, 3, 9, 5, 19, 54, tzinfo=<UTC>),
 datetime.datetime(2015, 3, 25, 5, 19, 45, tzinfo=<UTC>),
 datetime.datetime(2015, 4, 10, 5, 19, 36, tzinfo=<UTC>),
 datetime.datetime(2015, 5, 28, 5, 19, 42, tzinfo=<UTC>),
 datetime.datetime(2015, 6, 13, 5, 19, 30, tzinfo=<UTC>),
 datetime.datetime(2015, 8, 16, 5, 37, 6, tzinfo=<UTC>),
 datetime.datetime(2015, 8, 16, 5, 37, 6, tzinfo=<UTC>),
 datetime.datetime(2015, 9, 5, 5, 37, 7, tzinfo=<UTC>),
 datetime.datetime(2015, 9, 17, 5, 20, 35, tzinfo=<UTC>),
 datetime.datetime(2015, 10, 3, 5, 20, 39, tzinfo=<UTC>),
 datetime.datetime(201